In [178]:
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import warnings
warnings.filterwarnings("ignore")

<h1 style="text-align:center">🏰 Data Modelling</h1> 

<h3 style="text-align:center">Predict BTC prices</h3> 

# 🎖 1. Get newest data points

## ➡️ Define symbol and time interval

In [179]:
symbol = "BTCUSDT"
PERIOD = "15m"

In [180]:
!jupyter nbconvert --execute --to notebook --inplace ../obtain/get_newest_price.ipynb

[NbConvertApp] Converting notebook ../obtain/get_newest_price.ipynb to notebook
[NbConvertApp] Writing 14782 bytes to ../obtain/get_newest_price.ipynb


In [181]:
pd_df = pd.read_csv(f"../../datastore/processed/{symbol}_{PERIOD}.csv")
pd_df = pd_df.iloc[-400:]
pd_df

,Kline open time,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
171936,1672970400000,16827.45,16830.15,16802.01,16821.32,3051.99031,1672971299999,5.132653e+07,65606,1405.61676,2.363902e+07
171937,1672971300000,16821.32,16835.31,16821.04,16825.78,1304.39170,1672972199999,2.195179e+07,39677,668.71189,1.125387e+07
171938,1672972200000,16825.78,16835.36,16821.02,16830.93,1432.35933,1672973099999,2.410500e+07,43381,731.98468,1.231856e+07
171939,1672973100000,16830.93,16839.79,16828.34,16837.06,1211.93775,1672973999999,2.040240e+07,36949,625.18019,1.052473e+07
171940,1672974000000,16836.65,16841.91,16827.00,16828.81,1200.94608,1672974899999,2.021535e+07,32790,613.15404,1.032115e+07
...,...,...,...,...,...,...,...,...,...,...,...
172331,1673325900000,17205.62,17222.50,17204.48,17221.68,2321.80383,1673326799999,3.997064e+07,52231,1197.94569,2.062377e+07
172332,1673326800000,17222.05,17224.06,17213.11,17217.24,1331.50172,1673327699999,2.292514e+07,40174,623.67993,1.073841e+07
172333,1673327700000,17216.95,17223.34,17208.00,17215.23,1684.91267,1673328599999,2.900936e+07,41384,813.47768,1.400594e+07
172334,1673328600000,17215.23,17225.99,17207.05,17221.85,1363.17022,1673329499999,2.347062e+07,37643,663.41403,1.142270e+07


## ➡️ Select features

### 📌 Using Open time as index to visualize later

In [182]:
pd_df["Open price"] = pd_df["Open price"].apply(lambda price: float(price))
pd_df = pd_df.set_index("Kline open time").sort_index() 
pd_df.index = pd.to_datetime(pd_df.index, unit="ms") + pd.Timedelta('07:00:00')
pd_df

,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
Kline open time,,,,,,,,,,
2023-01-06 09:00:00,16827.45,16830.15,16802.01,16821.32,3051.99031,1672971299999,5.132653e+07,65606,1405.61676,2.363902e+07
2023-01-06 09:15:00,16821.32,16835.31,16821.04,16825.78,1304.39170,1672972199999,2.195179e+07,39677,668.71189,1.125387e+07
2023-01-06 09:30:00,16825.78,16835.36,16821.02,16830.93,1432.35933,1672973099999,2.410500e+07,43381,731.98468,1.231856e+07
2023-01-06 09:45:00,16830.93,16839.79,16828.34,16837.06,1211.93775,1672973999999,2.040240e+07,36949,625.18019,1.052473e+07
2023-01-06 10:00:00,16836.65,16841.91,16827.00,16828.81,1200.94608,1672974899999,2.021535e+07,32790,613.15404,1.032115e+07
...,...,...,...,...,...,...,...,...,...,...
2023-01-10 11:45:00,17205.62,17222.50,17204.48,17221.68,2321.80383,1673326799999,3.997064e+07,52231,1197.94569,2.062377e+07
2023-01-10 12:00:00,17222.05,17224.06,17213.11,17217.24,1331.50172,1673327699999,2.292514e+07,40174,623.67993,1.073841e+07
2023-01-10 12:15:00,17216.95,17223.34,17208.00,17215.23,1684.91267,1673328599999,2.900936e+07,41384,813.47768,1.400594e+07


### 📌 Visualize prices as candle stick

In [183]:
fig = go.Figure(data=[go.Candlestick(x=pd_df.index,
                open=pd_df['Open price'],
                high=pd_df['High price'],
                low=pd_df['Low price'],
                close=pd_df['Close price'])])
# set new height and width
fig.update_layout(
    height=800,
    width=1000,
    title_text="BTC/USDT price",
    yaxis_title="Price (BTC/USDT)",
    xaxis_title="Date",
    xaxis_rangeslider_visible=True
)

fig.show()

### 📌 Using open price as feature and target as well

In [184]:
dataset = pd_df.filter(["Open price"]).values
dataset[:5]

array([[16827.45],
       [16821.32],
       [16825.78],
       [16830.93],
       [16836.65]])

# 🎖 2. Prepare train-test set

✅ Train-Test ratio: `80%` train, `20%` test <br>
✅ Train-Valid ratio: `70%` train, `30%` valid

In [185]:
TRAIN_TEST_LENGTH = int(len(dataset) * 0.8)
TRAIN_VALID_LENGTH = int(TRAIN_TEST_LENGTH * 0.7)

## ➡️ Scale data

In [186]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)
scaled_data[:5]

array([[0.1991965 ],
       [0.19040103],
       [0.19680034],
       [0.20418968],
       [0.21239687]])

## ➡️ Prepare time series data

In [187]:
def get_split_data(start, end = None):
    data = scaled_data[start:end]
    X = []
    Y = []
    for i in range(60,len(data)):
        X.append(data[i-60:i,:])
        Y.append(data[i,:])

    X, Y = np.array(X), np.array(Y)
    return X,Y

## ➡️ Split train-test set

In [188]:
X_origin_train, Y_origin_train = get_split_data(0, TRAIN_TEST_LENGTH)
X_test, Y_test = get_split_data(TRAIN_TEST_LENGTH-60, )

print(X_origin_train.shape, Y_origin_train.shape)
print(X_test.shape, Y_test.shape)

(260, 60, 1) (260, 1)
(80, 60, 1) (80, 1)


## ➡️ Split train-valid set

In [189]:
X_train, Y_train = get_split_data(0, TRAIN_VALID_LENGTH)
X_valid, Y_valid = get_split_data(TRAIN_VALID_LENGTH-60, TRAIN_TEST_LENGTH)

print(X_train.shape, Y_train.shape)
print(X_valid.shape, Y_valid.shape)

(164, 60, 1) (164, 1)
(96, 60, 1) (96, 1)


# 🎖 3. Naive training

## ➡️ Visualize predictions

In [190]:
def visualize_prediction(preds, variant="train"):
    if variant == "train":
        train_end = TRAIN_VALID_LENGTH
        valid_start = TRAIN_VALID_LENGTH
        valid_end = TRAIN_TEST_LENGTH
    else:
        train_end = TRAIN_TEST_LENGTH
        valid_start = TRAIN_TEST_LENGTH
        valid_end = None
        
    
    data = pd_df[["Open price"]]
    train = data[:train_end]
    valid = data[valid_start:valid_end].reset_index()
    valid["Predict"] = preds
    
    if variant == "test":
        print(valid)
        
    valid = valid.set_index("Kline open time")
    valid["Predict"]= valid["Predict"].apply(lambda price: float(price))

    concat_df = pd.concat([train, valid], axis=0)
    fig = px.line(concat_df[["Open price","Predict"]], title="BTC/USDT price" , width=1000, height=800)
    fig.show()

In [191]:
X_train.shape

(164, 60, 1)

## ➡️ Linear Regression

### 📌 Build model and train

In [192]:
from sklearn.linear_model import LinearRegression

In [217]:
model = LinearRegression()
model.fit(X_train.reshape(X_train.shape[0], -1), Y_train)

LinearRegression()

### 📌 Predict and transform to the original scale

In [194]:
predictions = model.predict(X_valid.reshape(X_valid.shape[0], -1))
predictions = scaler.inverse_transform(np.array(predictions))
predictions.shape

(96, 1)

### 📌 Visualize the prediction

In [195]:
visualize_prediction(predictions)

## ➡️ Gated Recurrent Unit (GRU)

### 📌 Build model and train

In [196]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU

In [197]:
model = Sequential()
model.add(GRU(200, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(GRU(100))
model.add(Dense(X_train.shape[2]))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, Y_train, batch_size=1, epochs=1)

164/164 [==============================] - 4s 13ms/step - loss: 0.0021


### 📌 Predict and transform to the original scale

In [198]:
predictions = model.predict(X_valid)
predictions = scaler.inverse_transform(predictions)
predictions.shape

3/3 [==============================] - 0s 31ms/step


(96, 1)

### 📌 Visualize the prediction

In [199]:
visualize_prediction(predictions)

## ➡️ Seasonal Auto Regressive Integrated Moving Average (SARIMAX)

In [200]:
from statsmodels.tsa.api import SARIMAX

In [201]:
dataset = pd_df.filter(["Open price"]).values
dataset[:5]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)
scaled_data[:5]
X_SARIMAX_valid, Y_SARIMAX_valid = get_split_data(TRAIN_VALID_LENGTH-60, TRAIN_TEST_LENGTH)

### 📌 Build model and train

In [ ]:
predictions = list()
for t in range(X_SARIMAX_valid.shape[0]):
    model = SARIMAX(X_SARIMAX_valid[t])
    model_fit = model.fit()
    output = model_fit.forecast()
    pred_price = output[0]
    predictions.append(pred_price)

### 📌 Predict and transform to the original scale

In [203]:
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
predictions.shape

(96, 1)

### 📌 Visualize the prediction

In [204]:
visualize_prediction(predictions)        

# 🎖 4. Model selection and evaluation

In [205]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, r2_score
from estimator import LSTMEstimator, SarimaxEstimator, LinearRegressionEstimator

## ➡️ Naive cross validation to select model

### 📌 Cross validation method

In [206]:
tscv = TimeSeriesSplit(n_splits=3)

- The method that can be used for cross-validating the time-series model is cross-validation on a rolling basis. Start with a small subset of data for training purpose, forecast for the later data points and then checking the accuracy for the forecasted data points. The same forecasted data points are then included as part of the next training dataset and subsequent data points are forecasted. <br> <br>
![](https://miro.medium.com/max/640/1*XcqvKVTQ6U_zszSD52lSqA.webp)

### 📌 Metric method

In [207]:
metric = make_scorer(r2_score)

- $R^2$ score is used to evaluate the performance of a linear regression model. It is the amount of the variation in the output dependent attribute which is predictable from the input independent variable(s) <br> <br>
![](https://vitalflux.com/wp-content/uploads/2019/07/R-squared-formula-function-of-SSE-and-SST.jpg)

### 📌 Evaluate models

In [208]:
estimators = [
    LinearRegressionEstimator(),
    LSTMEstimator(), 
    SarimaxEstimator(), 
]

In [ ]:
scores_dict = {}

for estimator in estimators:
    scores = cross_val_score(estimator, X_origin_train, Y_origin_train, scoring=metric, cv=tscv, n_jobs=-1, verbose=3)
    print(estimator.__class__.__name__, scores.mean())
    scores_dict[estimator.__class__.__name__] = scores
    
scores_df = pd.DataFrame(scores_dict)
scores_df.index = scores_df.index.map(lambda x: f'iter_{x+1}')
scores_df.loc['mean'] = scores_df.mean()

### 📌 Score and reveal best model

In [210]:
scores_df

,LinearRegressionEstimator,LSTMEstimator,SarimaxEstimator
iter_1,-17.879112,-1.323579,0.485217
iter_2,0.008336,-0.890989,-0.017546
iter_3,0.905439,0.301900,0.957004
mean,-5.655112,-0.637556,0.474892


- The best best_estimator is one that have highest mean of scores

In [211]:
best_estimator = scores_df.loc['mean'].idxmin()
best_estimator

'LinearRegressionEstimator'

## ➡️ Select best model with best hyperparameters using grid search

In [251]:
from estimator import Estimator, LinearRegressionEstimator, SarimaxEstimator, LSTMEstimator
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline

### 📌 Cross validation and metric method

- Using aforementioned methods

In [252]:
NUM_SPLIT = 5

In [253]:
tscv = TimeSeriesSplit(n_splits=NUM_SPLIT)
metric = make_scorer(r2_score)

### 📌 Evaluate hyperparameters and models 

In [254]:
parameters = [
    {
        'reg__estimator': [LinearRegressionEstimator()],
        'reg__estimator__fit_intercept': [True, False],
    },
    {
        'reg__estimator': [SarimaxEstimator()],
        'reg__estimator__order': [(1, 1, 1), (3, 1, 1)],
        'reg__estimator__seasonal_order': [(1, 1, 1, 12), (3, 1, 1, 12)]
    },
    {
        'reg__estimator': [LSTMEstimator()],
        'reg__estimator__epochs': [1, 5, 10],
        'reg__estimator__batch_size': [4, 16, 32],
        'reg__estimator__neurons': [50, 100, 200]
    },
]

In [ ]:
pipeline = Pipeline(
    steps=[("reg", Estimator())]
)

grid_search = GridSearchCV(pipeline, parameters, scoring=metric, cv=tscv, n_jobs=2, verbose=3)
grid_search.fit(X_train, Y_train)

- Best model with best hyperparameters

In [256]:
def GridSearch_table_plot(grid_clf, param_name,
                          num_results=15,
                          negative=True,
                          display_all_params=True):
    clf = grid_clf.best_estimator_
    clf_params = grid_clf.best_params_
    if negative:
        clf_score = -grid_clf.best_score_
    else:
        clf_score = grid_clf.best_score_
    clf_stdev = grid_clf.cv_results_['std_test_score'][grid_clf.best_index_]
    cv_results = grid_clf.cv_results_

    print("best parameters: {}".format(clf_params))
    print("best score:      {:0.5f} (+/-{:0.5f})".format(clf_score, clf_stdev))
    if display_all_params:
        import pprint
        pprint.pprint(clf.get_params())

    # pick out the best results
    # =========================
    scores_df = pd.DataFrame(cv_results).sort_values(by='rank_test_score')

    best_row = scores_df.iloc[0, :]
    if negative:
        best_mean = -best_row['mean_test_score']
    else:
        best_mean = best_row['mean_test_score']
    best_stdev = best_row['std_test_score']
    best_param = best_row['param_' + param_name]

    # display the top 'num_results' results
    result_table = pd.DataFrame(cv_results).sort_values(by='rank_test_score').head(num_results)
    result_table.to_csv(f'../../datastore/model_scores/grid_search_cv_{NUM_SPLIT}_len_{dataset.shape[0]}.csv', index=False)
    display(result_table)

    # plot the results
    # scores_df = scores_df.sort_values(by='param_' + param_name)
    if negative:
        means = -scores_df['mean_test_score']
    else:
        means = scores_df['mean_test_score']
    stds = scores_df['std_test_score']
    params = scores_df['param_' + param_name]
        
GridSearch_table_plot(grid_search, "reg__estimator", negative=False)

best parameters: {'reg__estimator': SarimaxEstimator(order=(1, 1, 1)), 'reg__estimator__order': (1, 1, 1), 'reg__estimator__seasonal_order': (1, 1, 1, 12)}
best score:      -0.04408 (+/-0.44518)
{'memory': None,
 'reg': Estimator(estimator=SarimaxEstimator(order=(1, 1, 1))),
 'reg__estimator': SarimaxEstimator(order=(1, 1, 1)),
 'reg__estimator__order': (1, 1, 1),
 'reg__estimator__seasonal_order': (1, 1, 1, 12),
 'steps': [('reg', Estimator(estimator=SarimaxEstimator(order=(1, 1, 1))))],
 'verbose': False}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_reg__estimator,param_reg__estimator__order,param_reg__estimator__seasonal_order,param_reg__estimator__batch_size,param_reg__estimator__epochs,param_reg__estimator__neurons,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,0.000384,0.000178,9.531185,1.005062,"SarimaxEstimator(order=(1, 1, 1))","(1, 1, 1)","(1, 1, 1, 12)",NaN,NaN,NaN,"{'reg__estimator': SarimaxEstimator(order=(1, ...",-0.777588,0.438430,0.402843,-0.166042,-0.118045,-0.044080,0.445182,1
3,0.000279,0.000075,11.805284,0.893188,"SarimaxEstimator(order=(1, 1, 1))","(3, 1, 1)","(1, 1, 1, 12)",NaN,NaN,NaN,"{'reg__estimator': SarimaxEstimator(order=(1, ...",-1.135629,0.367213,0.425071,-0.134021,-0.107813,-0.117036,0.559693,2
22,5.744035,1.019852,0.691727,0.347146,LSTMEstimator(),NaN,NaN,16,10,200,"{'reg__estimator': LSTMEstimator(), 'reg__esti...",0.271068,-1.572680,-0.186916,0.190466,0.244140,-0.210784,0.700781,3
20,3.818820,0.451015,0.588219,0.122569,LSTMEstimator(),NaN,NaN,16,10,50,"{'reg__estimator': LSTMEstimator(), 'reg__esti...",-0.629153,-1.169812,-0.720388,-0.249169,-0.045314,-0.562767,0.390860,4
2,0.000307,0.000094,35.379433,2.517851,"SarimaxEstimator(order=(1, 1, 1))","(1, 1, 1)","(3, 1, 1, 12)",NaN,NaN,NaN,"{'reg__estimator': SarimaxEstimator(order=(1, ...",-2.825261,-0.888557,0.382177,-0.320446,0.155181,-0.699381,1.148807,5
4,0.000248,0.000051,42.971854,4.679828,"SarimaxEstimator(order=(1, 1, 1))","(3, 1, 1)","(3, 1, 1, 12)",NaN,NaN,NaN,"{'reg__estimator': SarimaxEstimator(order=(1, ...",-2.848606,-0.818736,0.281822,-0.284553,-0.050904,-0.744196,1.111642,6
9,4.518883,1.216331,0.497264,0.083281,LSTMEstimator(),NaN,NaN,4,5,100,"{'reg__estimator': LSTMEstimator(), 'reg__esti...",-4.361688,-0.789072,-1.666060,-1.112055,-0.000155,-1.585806,1.488991,7
11,6.246914,1.997236,0.492712,0.082086,LSTMEstimator(),NaN,NaN,4,10,50,"{'reg__estimator': LSTMEstimator(), 'reg__esti...",-7.088249,-2.170862,-2.120874,0.104918,-0.950621,-2.445138,2.468814,8
10,6.060584,2.120379,0.563912,0.101262,LSTMEstimator(),NaN,NaN,4,5,200,"{'reg__estimator': LSTMEstimator(), 'reg__esti...",-11.314687,0.374141,-0.396718,0.172975,-2.249958,-2.682849,4.414144,9
19,4.267075,0.564000,0.575039,0.102347,LSTMEstimator(),NaN,NaN,16,5,200,"{'reg__estimator': LSTMEstimator(), 'reg__esti...",-6.552123,0.230125,-2.346671,-5.600857,0.147503,-2.824405,2.828257,10


# 🎖 5. Testing

### 📌 Evaluate with test set

In [ ]:
predictions = []
for t in range(X_test.shape[0]):
    model = SARIMAX(X_test[t], seasonal_order=(1, 1, 1, 12), order=(1, 1, 1))
    model_fit = model.fit()
    output = model_fit.forecast()
    pred_price = output[0]
    predictions.append(pred_price)

### 📌 Metric method

In [278]:
r2_score(Y_test, np.array(predictions).reshape(-1, 1))

0.8450086063783957

- ✅ Incredible score

### 📌 Visualize result

In [279]:
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

In [280]:
visualize_prediction(predictions, variant="test")

       Kline open time  Open price       Predict
0  2023-01-09 17:00:00    17238.38  17245.139643
1  2023-01-09 17:15:00    17245.76  17239.457647
2  2023-01-09 17:30:00    17261.63  17258.577955
3  2023-01-09 17:45:00    17270.25  17264.361020
4  2023-01-09 18:00:00    17264.10  17265.362766
..                 ...         ...           ...
75 2023-01-10 11:45:00    17205.62  17213.394444
76 2023-01-10 12:00:00    17222.05  17193.037693
77 2023-01-10 12:15:00    17216.95  17196.625626
78 2023-01-10 12:30:00    17215.23  17216.117134
79 2023-01-10 12:45:00    17221.85  17205.112475

[80 rows x 3 columns]


# 🎖 5. Predict future prices

## ➡️ Utilize the best model to predict prices

In [ ]:
from statsmodels.tsa.api import SARIMAX

data = pd_df.filter(["Open price"])

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data).reshape(-1)

predictions = list()
model = SARIMAX(scaled_data, order=(1,1,1), seasonal_order=(1,1,1,12))
model_fit = model.fit()
output = model_fit.forecast()
pred_price = output[0]
predictions.append(pred_price)

In [282]:
pred_price = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

## ➡️ Next price

In [283]:
strtime = (pd_df.index[-1] + pd.Timedelta('00:15:00')).strftime("%Y-%m-%d %H:%M")
print(f"Predict price at {strtime} is {pred_price[0][0]}")

Predict price at 2023-01-10 13:00 is 17222.245033831616
